<a href="https://colab.research.google.com/github/Shreyas5848/Household-energy-consumption/blob/main/Gaussian-Mixture-Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('/content/KwhConsumptionBlower78_1 (1).csv')
data

,Unnamed: 0,TxnDate,TxnTime,Consumption
0,76229,01 Jan 2022,16:55:52,1.010
1,76258,01 Jan 2022,21:45:29,0.908
2,76287,01 Jan 2022,12:24:52,0.926
3,76316,01 Jan 2022,04:07:36,0.928
4,76345,01 Jan 2022,06:52:25,0.916
...,...,...,...,...
1223,97922,20 Jan 2022,21:43:55,5.128
1224,97951,20 Jan 2022,09:43:03,5.924
1225,97980,20 Jan 2022,03:06:05,4.664
1226,98009,20 Jan 2022,06:16:38,3.474


In [3]:
data['Datetime'] = pd.to_datetime(data['TxnDate'] + ' ' + data['TxnTime'], format='%d %b %Y %H:%M:%S')
data['Date'] = data['Datetime'].dt.date


In [4]:
data = data.drop(['TxnDate','TxnTime'],axis=1)
data = data.drop(['Unnamed: 0'], axis=1)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1228 entries, 0 to 1227
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Consumption  1228 non-null   float64       
 1   Datetime     1228 non-null   datetime64[ns]
 2   Date         1228 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 28.9+ KB


In [6]:
data.describe()

,Consumption,Datetime
count,1228.000000,1228
mean,3.428194,2022-01-10 23:42:46.149022720
min,0.842000,2022-01-01 00:22:28
25%,0.952000,2022-01-06 03:18:53.750000128
50%,1.121000,2022-01-10 22:29:25
75%,5.754500,2022-01-15 19:39:20.249999872
max,27.108000,2022-01-20 23:47:55
std,3.535620,NaN


In [7]:
daily_data = data.groupby('Date')['Consumption'].agg(['sum', 'mean', 'max', 'std']).reset_index()
daily_data.columns = ['Date', 'TotalConsumption', 'AvgConsumption', 'PeakConsumption', 'ConsumptionStdDev']

In [10]:
from sklearn.preprocessing import StandardScaler

In [11]:
features = daily_data[['TotalConsumption', 'AvgConsumption', 'PeakConsumption', 'ConsumptionStdDev']]
scaler = StandardScaler()
normalized_features = scaler.fit_transform(features)

In [12]:
from sklearn.mixture import GaussianMixture

In [13]:
gmm = GaussianMixture(n_components=3, random_state=42)
gmm_labels = gmm.fit_predict(normalized_features)


In [14]:
daily_data['Cluster'] = gmm_labels

In [16]:
from sklearn.decomposition import PCA

In [17]:
pca = PCA(n_components=2)
reduced_features = pca.fit_transform(normalized_features)


In [18]:
pca_df = pd.DataFrame(reduced_features, columns=['PCA1', 'PCA2'])
pca_df['Cluster'] = gmm_labels

In [24]:
fig = px.scatter(pca_df, x='PCA1', y='PCA2', color='Cluster', title="GMM Clusters (PCA Visualization)",
                 labels={'color': 'Cluster'})
fig.show()

In [26]:
cluster_summary = daily_data.groupby('Cluster').agg({'TotalConsumption': 'mean','AvgConsumption': 'mean','PeakConsumption': 'mean',
                                                     'ConsumptionStdDev': 'mean'})
print(cluster_summary)

         TotalConsumption  AvgConsumption  PeakConsumption  ConsumptionStdDev
Cluster                                                                      
0               65.726800        1.052717         2.165200           0.288183
1              321.672222        5.319482        18.164000           3.601438
2              164.356333        2.618119        15.938667           3.135320


In [30]:
from sklearn.metrics import silhouette_score

silhouette_avg = silhouette_score(normalized_features, gmm_labels)
print(f'Silhouette Score: {silhouette_avg}')

Silhouette Score: 0.5495745416264106
